<a href="https://colab.research.google.com/github/RafsanJany-44/Machine-School/blob/main/Particle_Swarm_Optimizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
"""from google.colab import drive
drive.mount('/content/drive')"""

"from google.colab import drive\ndrive.mount('/content/drive')"

In [11]:
from sklearn.datasets import load_breast_cancer

In [12]:
dataset = load_breast_cancer()
X = dataset.data
y = dataset.target
feature_names = dataset.feature_names

In [13]:
dataset.feature_names

array(['mean radius', 'mean texture', 'mean perimeter', 'mean area',
       'mean smoothness', 'mean compactness', 'mean concavity',
       'mean concave points', 'mean symmetry', 'mean fractal dimension',
       'radius error', 'texture error', 'perimeter error', 'area error',
       'smoothness error', 'compactness error', 'concavity error',
       'concave points error', 'symmetry error',
       'fractal dimension error', 'worst radius', 'worst texture',
       'worst perimeter', 'worst area', 'worst smoothness',
       'worst compactness', 'worst concavity', 'worst concave points',
       'worst symmetry', 'worst fractal dimension'], dtype='<U23')

In [14]:
!pip install imbalanced-learn

from imblearn.over_sampling import SMOTE

def balance(X_temp, y_temp):
  smote = SMOTE()
  X_temp, y_temp= smote.fit_resample(X_temp, y_temp)
  return pd.concat([pd.DataFrame(X_temp), pd.DataFrame(y_temp)], axis=1)



def models_check_box(models):
  import ipywidgets as widgets
  from IPython.display import display
  new_keys=[]
  for i in models:
    i=widgets.Checkbox(
      value=False,
      description=str(i),
      disabled=False,
      indent=False
      )
    display(i)
    new_keys.append(i)
  return new_keys

In [15]:
import numpy as np
import pandas as pd
#dataset=pd.read_csv("/content/drive/MyDrive/Sleep Stage XAI/EEG_HMC_FeatureExtraction_2023.01.19.csv")
#dataset=pd.read_excel("/content/EEG_CNU_Control_Resting, walking, working and Reading_2023.01.07.xlsx")

#dataset=pd.read_csv("/content/drive/MyDrive/Cornell/MADE Lab/Wearable Project/Data/Fitbit ML/Feature merge/Fitbit_feature_merged_version_2023.06.26_ST-II&ST-V_Pre.csv")

dataset = pd.read_csv("/content/Fitbit_feature_merged_version_2023.06.26_ST-II&ST-V_Pre.csv")

target = "Activity"
result = {}

In [16]:
dataset.dropna(axis=0, how='any', inplace = True)
dataset = dataset.reset_index(drop=True)

In [17]:
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
dataset[target]=encoder.fit_transform(dataset[target])

In [18]:
X = dataset.iloc[:,11:]

y = dataset[target]

X.head(10)


,nni_counter,nni_mean,nni_min,nni_max,hr_mean,hr_min,hr_max,hr_std,nni_diff_mean,nni_diff_min,...,ar_norm_high,ar_ratio,ar_total,sd1,sd2,sd_ratio,ellipse_area,sampen,dfa_alpha1,dfa_alpha2
0,65,779.015385,740,810,77.047607,74.074074,81.081081,1.459607,6.078125,0,...,64.958716,0.539439,19105.45764,6.012997,19.842689,3.299967,374.836049,1.022451,1.470755,0.573684
1,65,703.938462,666,740,85.275756,81.081081,90.090090,1.890121,7.453125,0,...,64.912631,0.540532,18856.43483,6.145682,21.024745,3.421060,405.929592,1.240787,1.234339,1.042289
2,66,774.393939,731,833,77.537793,72.028812,82.079343,2.133033,9.107692,0,...,64.862213,0.541730,19039.04728,8.357614,28.930985,3.461632,759.618229,1.043804,1.534029,1.049467
3,77,762.038961,697,821,78.878037,73.081608,86.083214,3.387898,10.381579,0,...,65.007546,0.538283,18767.08842,9.978556,44.186165,4.428112,1385.172625,1.504077,1.542662,0.948630
4,72,688.861111,652,731,87.146588,82.079343,92.024540,2.014072,7.098592,0,...,64.868522,0.541580,18800.46155,6.527922,21.178959,3.244365,434.339619,1.376244,1.358871,0.791963
5,72,735.500000,697,800,81.660141,75.000000,86.083214,2.589556,7.929577,0,...,64.795921,0.543307,18778.69359,7.778589,32.874130,4.226233,803.350301,1.712979,1.579461,0.753810
6,66,743.106061,681,789,80.820900,76.045627,88.105727,2.573095,6.615385,0,...,64.784223,0.543586,18906.56635,6.443601,31.820584,4.938323,644.149531,1.029619,1.548781,0.602646
7,72,642.000000,612,674,93.517353,89.020772,98.039216,2.372689,5.971831,0,...,64.778278,0.543727,18563.27065,5.375892,21.911972,4.075969,370.068303,1.252763,1.407493,-0.192170
8,72,655.638889,582,697,91.598758,86.083214,103.092783,2.891687,6.211268,0,...,65.099660,0.536106,18641.14134,7.500777,26.517596,3.535313,624.870861,1.134980,1.640218,0.796802
9,75,681.346667,618,714,88.188794,84.033613,97.087379,3.439274,7.148649,0,...,64.914935,0.540478,18613.09652,6.361041,35.605096,5.597370,711.525070,1.293921,1.492883,2.042574


In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [20]:
!pip install niapy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.9/181.9 kB 16.1 MB/s eta 0:00:00


In [21]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.svm import SVC

from niapy.problems import Problem
from niapy.task import Task
from niapy.algorithms.basic import ParticleSwarmOptimization


class SVMFeatureSelection(Problem):
    def __init__(self, X_train, y_train, alpha=0.99):
        super().__init__(dimension=X_train.shape[1], lower=0, upper=1)
        self.X_train = X_train
        self.y_train = y_train
        self.alpha = alpha

    def _evaluate(self, x):
        selected = x > 0.5
        num_selected = selected.sum()
        if num_selected == 0:
            return 1.0
        accuracy = cross_val_score(SVC(), self.X_train[:, selected], self.y_train, cv=2, n_jobs=-1).mean()
        score = 1 - accuracy
        num_features = self.X_train.shape[1]
        return self.alpha * score + (1 - self.alpha) * (num_selected / num_features)

In [22]:
X.columns

Index(['nni_counter', 'nni_mean', 'nni_min', 'nni_max', 'hr_mean', 'hr_min',
       'hr_max', 'hr_std', 'nni_diff_mean', 'nni_diff_min', 'nni_diff_max',
       'sdnn', 'rmssd', 'sdsd', 'nn50', 'pnn50', 'nn20', 'pnn20', 'tinn_n',
       'tinn_m', 'tinn', 'tri_index', 'fft_peak_low', 'fft_peak_medium',
       'fft_peak_high', 'fft_abs_low', 'fft_abs_medium', 'fft_abs_high',
       'fft_rel_low', 'fft_rel_medium', 'fft_log_low', 'fft_log_medium',
       'fft_log_high', 'fft_norm_low', 'fft_norm_high', 'fft_ratio',
       'fft_total', 'lomb_peak_low', 'lomb_peak_medium', 'lomb_peak_high',
       'lomb_abs_low', 'lomb_abs_medium', 'lomb_abs_high', 'lomb_rel_low',
       'lomb_rel_medium', 'lomb_rel_high', 'lomb_log_low', 'lomb_log_medium',
       'lomb_log_high', 'lomb_norm_high', 'lomb_norm_Low', 'lomb_ratio',
       'lomb_total', 'ar_abs_low', 'ar_abs_medium', 'ar_abs_high',
       'ar_rel_low', 'ar_rel_medium', 'ar_rel_high', 'ar_log_low',
       'ar_log_medium', 'ar_log_high', 'ar_norm_

In [23]:
X_train = X_train.values
X_test = X_test.values
y_train = y_train.values
y_test = y_test.values

problem = SVMFeatureSelection(X_train, y_train)
task = Task(problem, max_iters=100)
algorithm = ParticleSwarmOptimization(population_size=10, seed=1234)
best_features, best_fitness = algorithm.run(task)

selected_features = best_features > 0.5
print('Number of selected features:', selected_features.sum())
print('Selected features:', ', '.join(X.columns[selected_features].tolist()))

model_selected = SVC()
model_all = SVC()

model_selected.fit(X_train[:, selected_features], y_train)
print('Subset accuracy:', model_selected.score(X_test[:, selected_features], y_test))

model_all.fit(X_train, y_train)
print('All Features Accuracy:', model_all.score(X_test, y_test))

Number of selected features: 30
Selected features: nni_counter, nni_mean, nni_min, hr_mean, hr_max, nni_diff_min, rmssd, nn20, tinn_n, tinn_m, fft_peak_low, fft_peak_medium, fft_abs_medium, fft_log_low, fft_log_medium, lomb_peak_low, lomb_peak_medium, lomb_abs_low, lomb_abs_medium, lomb_abs_high, lomb_rel_low, lomb_rel_medium, lomb_log_low, lomb_norm_Low, lomb_ratio, ar_log_low, sd1, sd2, sd_ratio, dfa_alpha1
Subset accuracy: 0.8783269961977186
All Features Accuracy: 0.8212927756653993


In [24]:
selected_features

array([ True,  True,  True, False,  True, False,  True, False, False,
        True, False, False,  True, False, False, False,  True, False,
        True,  True, False, False,  True,  True, False, False,  True,
       False, False, False,  True,  True, False, False, False, False,
       False,  True,  True, False,  True,  True,  True,  True,  True,
       False,  True, False, False, False,  True,  True, False, False,
       False, False, False, False, False,  True, False, False, False,
       False, False, False,  True,  True,  True, False, False,  True,
       False])

array([1.        , 1.        , 1.        , 0.        , 1.        ,
       0.29603973, 1.        , 0.        , 0.        , 1.        ,
       0.        , 0.        , 1.        , 0.        , 0.        ,
       0.        , 1.        , 0.0600817 , 1.        , 1.        ,
       0.        , 0.42971966, 1.        , 0.91198598, 0.        ,
       0.01916841, 1.        , 0.        , 0.        , 0.49689507,
       1.        , 1.        , 0.        , 0.        , 0.        ,
       0.        , 0.25390792, 1.        , 1.        , 0.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ,
       0.        , 1.        , 0.        , 0.        , 0.30844251,
       1.        , 1.        , 0.        , 0.        , 0.47353937,
       0.        , 0.        , 0.        , 0.40067164, 1.        ,
       0.        , 0.08231331, 0.        , 0.        , 0.        ,
       0.        , 1.        , 1.        , 1.        , 0.        ,
       0.        , 1.        , 0.        ])